## <span style="color: #336699">Parte 05 - Comparações e plots usando o ACOLITE e o Sen2COR </span>
<hr style="border:0.5px solid #0077b9;">


Agora que conseguimos baixar e rodar a correção atmosférica com o ACOLITE, podemos aplicar o modelo de Chl-a na imagem e avaliar os resultados com ela também.


<br>



In [ ]:
require(data.table)
require(dplyr)
require(rstac)
require(terra)
require(httr)
library(rasterVis)
require(RColorBrewer)

options(warn = -1)
options(repr.plot.width = 20, repr.plot.height = 10) 


In [ ]:
# Carregando o modelo de NDCI
modelo_ndci = readRDS('Outputs/emp_chla.R')

In [ ]:
# Carregando os resultados do Sen2Cor

chla_sen2cor = rast("Outputs/chla_sen2cor.tif")

names(chla_sen2cor) = "Chla Sen2Cor"

In [ ]:
## Carregando o ACOLITE

b5_acolite = rast("Outputs/ACOLITE/S2B_MSIL1C_20250306T131249_N0511_R138_T23KLP_20250306T144532.SAFE//S2B_MSI_2025_03_06_13_18_59_T23KLP_L2W_Rrs_704.tif")
b4_acolite = rast("Outputs/ACOLITE/S2B_MSIL1C_20250306T131249_N0511_R138_T23KLP_20250306T144532.SAFE//S2B_MSI_2025_03_06_13_18_59_T23KLP_L2W_Rrs_665.tif")
b3_acolite = rast("Outputs/ACOLITE/S2B_MSIL1C_20250306T131249_N0511_R138_T23KLP_20250306T144532.SAFE//S2B_MSI_2025_03_06_13_18_59_T23KLP_L2W_Rrs_559.tif")
b2_acolite = rast("Outputs/ACOLITE/S2B_MSIL1C_20250306T131249_N0511_R138_T23KLP_20250306T144532.SAFE//S2B_MSI_2025_03_06_13_18_59_T23KLP_L2W_Rrs_492.tif")


# Mosaico
mosaico = c(b2_acolite, b3_acolite, b4_acolite, b5_acolite)

# Cálculo do NDCI
ndci_acolite = ((b5_acolite-b4_acolite)/(b5_acolite+b4_acolite))+1
names(ndci_acolite)[1] = "NDCI"

# Cálculo da concentração de clorofila-a

chla_acolite = predict(ndci_acolite, modelo_ndci, na.rm = T)

names(chla_acolite) = "Chla ACOLITE"


In [ ]:

plotRGB(mosaico, r = 3, g = 2, b = 1, stretch = 'lin')



In [ ]:
# Escala de cores
colr <- colorRampPalette(rev(brewer.pal(11, 'RdBu')))

# Reprojetando para a mesma escala do ACOLITE
sen2cor = project(x = chla_sen2cor, y = chla_acolite)

# Colocndo no mesmo arquivo
rasters = c(chla_acolite, sen2cor)

#plot 
levelplot(raster::stack(rasters),col.regions = colr, layout = c(2, 1),
                    at = seq(0, 250, length = 100),
          maxpixels = 1e9,
          main = "Chlorophyll-a Concentration (ug/L)") 

In [ ]:
df = data.frame(acolite = values(chla_acolite), sen2cor = values(sen2cor)) %>% na.omit()

In [ ]:
require(ggplot2)
ggplot(df, aes(x = Chla.ACOLITE, y = Chla.Sen2Cor)) + 
                    geom_point() + 
                    theme_bw() +
                    scale_x_continuous(limits = c(0,600)) + 
                    scale_y_continuous(limits = c(0,600)) + geom_abline(slope = 1, intercept = 0)


## <span style="color: #336699">E agora o que fazer?????? </span>
<hr style="border:0.5px solid #0077b9;">




Como comentado - o ideal é sempre aplicar correções atmosféricas específicas para água. Nesse caso abordado - essa imagem - os resultados são muito diferentes. Assim, precisamos ter consciência de qual dado é melhor para nossa aplicação. Observamos que houve um resídual de correção atmosférica nos dados do Sen2Cor - que é resolvido com a subtração da banda do SWIR (Não mostrado aqui no curso). Porém, temos que ter claro que estamos comparando coisas iguais - ou seja, dados no mesmo formato - os dados do ACOLITE não possuem a correção resídual ou correção de glint. 

Assim, recomendamos novamente fortemente o uso de métodos específios para água para aplicação de correção atmosférica. 

<br>



